# A/B Testing

## Test of Means Analysis
Statistical calculation of whether the mean values of the treatment and control groups are the same

## T-Test
Give p-value - likelihood the actual difference between the means is zero.  
p-value < 0.05 (statistically significant)  
T-test is important when working in small data set

There are three types of t-tests you can use:  
1. Paired  
2. Equal variance  
3. Unequal variance  

In a random experiment you will usually assume that variances between the groups are different, so we’ll use an unequal variance t-test.


In [4]:
# load package
import pandas as pd

In [9]:
# Supper hero data
super_hero = pd.read_excel('superherodata.xlsx')

super_hero

,Name,Alignment,Intelligence,Strength,Speed,Durability,Power,Combat,Unnamed: 8,Hero,Rival,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Iron Man,good,100.000000,85.000000,58.000000,85.000000,100.000000,64.000000,NaN,Iron Man,Ultron,NaN,NaN,NaN,NaN
1,Captain America,good,63.000000,65.000000,35.000000,56.000000,46.000000,100.000000,NaN,Captain America,Red Skull,NaN,NaN,NaN,NaN
2,Hulk,good,88.000000,100.000000,47.000000,100.000000,41.000000,85.000000,NaN,Hulk,Abomination,NaN,NaN,NaN,NaN
3,Hawkeye,good,62.000000,12.000000,23.000000,14.000000,26.000000,80.000000,NaN,Hawkeye,Bullseye,NaN,NaN,NaN,NaN
4,Thor,good,69.000000,100.000000,92.000000,100.000000,100.000000,85.000000,NaN,Thor,Loki,NaN,NaN,NaN,NaN
5,Spider-Man,good,88.000000,55.000000,60.000000,74.000000,58.000000,85.000000,NaN,Spider-Man,Green Goblin,NaN,NaN,NaN,NaN
6,Average,NaN,78.333333,69.500000,52.500000,71.500000,61.833333,83.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Ultron,bad,95.000000,83.000000,42.000000,100.000000,100.000000,64.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Red Skull,bad,90.000000,30.000000,12.000000,14.000000,19.000000,80.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
useful_columns = list(super_hero.columns)[:8]
useful_columns

['Name',
 'Alignment',
 'Intelligence',
 'Strength',
 'Speed',
 'Durability',
 'Power',
 'Combat']

In [16]:
super_hero_clean_data = super_hero[useful_columns].dropna()
super_hero_clean_data

,Name,Alignment,Intelligence,Strength,Speed,Durability,Power,Combat
0,Iron Man,good,100.0,85.0,58.0,85.0,100.0,64.0
1,Captain America,good,63.0,65.0,35.0,56.0,46.0,100.0
2,Hulk,good,88.0,100.0,47.0,100.0,41.0,85.0
3,Hawkeye,good,62.0,12.0,23.0,14.0,26.0,80.0
4,Thor,good,69.0,100.0,92.0,100.0,100.0,85.0
5,Spider-Man,good,88.0,55.0,60.0,74.0,58.0,85.0
8,Ultron,bad,95.0,83.0,42.0,100.0,100.0,64.0
9,Red Skull,bad,90.0,30.0,12.0,14.0,19.0,80.0
10,Abomination,bad,85.0,80.0,53.0,90.0,55.0,95.0
11,Bullseye,bad,75.0,11.0,25.0,70.0,20.0,70.0


In [43]:
# group good vs bad means
average = super_hero_clean_data.groupby(['Alignment']).mean()
average

,Intelligence,Strength,Speed,Durability,Power,Combat
Alignment,,,,,,
bad,86.166667,49.333333,35.666667,67.833333,52.833333,65.500000
good,78.333333,69.500000,52.500000,71.500000,61.833333,83.166667


,Intelligence,Strength,Speed,Durability,Power,Combat
Alignment,,,,,,
bad,86.166667,49.333333,35.666667,67.833333,52.833333,65.5


In [5]:
# Randomized Design Tests
data = pd.read_csv('grocerywebsiteabtestdata.csv')
data.head()

,RecordID,IP Address,LoggedInFlag,ServerID,VisitPageFlag
0,1,39.13.114.2,1,2,0
1,2,13.3.25.8,1,1,0
2,3,247.8.211.8,1,1,0
3,4,124.8.220.3,0,3,0
4,5,60.10.192.7,0,2,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184588 entries, 0 to 184587
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   RecordID       184588 non-null  int64 
 1   IP Address     184588 non-null  object
 2   LoggedInFlag   184588 non-null  int64 
 3   ServerID       184588 non-null  int64 
 4   VisitPageFlag  184588 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 7.0+ MB
